In [1]:
from random import random
import numpy as np
from itertools import combinations
import math
import random

In [2]:
x = np.array([[[1, 2, 3],
               [4, 5, 6],
               [7, 8, 9]],
              [[-1, -2, -3],
               [-4, -5, -6],
               [-7, -8, -9]],
              [[1, 2, 3],
               [4, 5, 6],
               [7, 8, 9]]])
x.shape

(3, 3, 3)

In [3]:
def add_all(i, j, k, g):
    g[i, j, k] = 1
    g[i, k, j] = 1
    g[j, i, k] = 1
    g[j, k, i] = 1
    g[k, i, j] = 1
    g[k, j, i] = 1
    return g

In [4]:
# g: tensor
# n: dimension, or rank of the tensor
# p: probability of creating edges
# k: cliqque size
def generate_graph(num_vertices, pr, k):
    #
    g = np.array([np.array([np.array([0 for i in range(0, num_vertices)]) for i in range(num_vertices)]) for i in
                  range(num_vertices)])
    vec = np.array([0 for i in range(0, num_vertices)])
    # print(vec)
    # Set edges
    for i in range(num_vertices):
        for j in range(i + 1, num_vertices):
            for k in range(j + 1, num_vertices):
                a = np.random.rand()
                # print(a)
                if a < pr:
                    vec[i] += 1
                    vec[j] += 1
                    vec[k] += 1
                    add_all(i, j, k, g)
    # plant a clique
    
    clique_vertices = random.sample(range(num_vertices), k)
    print(clique_vertices)

    l = len(clique_vertices)
    
    for i in range(l):
        for j in range(i + 1):
            for k in range(j + 1):
                a = clique_vertices[i]
                # print(a)
                vec[a] += 1
                b = clique_vertices[j]
                # print(b)
                vec[b] += 1
                c = clique_vertices[k]
                # print(c)
                vec[c] += 1
                add_all(a, b, c, g)

    return g, vec, clique_vertices


In [5]:
# n = 4
# p = 0.5
# # res = generate_graph(n, p)

In [6]:
# G = res[0]
# G

In [7]:
# vector storing the flattened list of vertices
# V = res[1]
# V

In [8]:
# np.flip(np.argsort(V))  # sort the indices by the number of presence in hyperedgess in descending order

In [9]:
def remove_all(i, j, k, g):
    g[i, j, k] = 0
    g[i, k, j] = 0
    g[j, i, k] = 0
    g[j, k, i] = 0
    g[k, i, j] = 0
    g[k, j, i] = 0
    return g


In [10]:
# remove edge
def remove_edges(g, n_dim, vec, curr):
    vec[curr] = 0
    graph_copy = g.copy()
    for j in range(0, n_dim):
        for k in range(0, n_dim):
            if graph_copy[curr, j, k] == 1:
                vec[j] -= 1
                vec[k] -= 1
                remove_all(curr, j, k, graph_copy)
    return graph_copy

In [11]:
def find_num_edges(g, num_vertices):
    num_edges = 0
    for i in range(num_vertices):
        for j in range(i + 1, num_vertices):
            for k in range(j + 1, num_vertices):
                if g[i, j, k] == 1:
                    num_edges += 1
    return num_edges

In [12]:

def is_clique(g, vec, num_vertices):

    active_count = np.count_nonzero(vec)
    edge_sum = find_num_edges(g, num_vertices)
    if edge_sum == 6 * math.comb(active_count, 3):
        return True
    return False

In [13]:
n = 4
p = 0.5
k = 3

res = generate_graph(n, p, k)
G = res[0]
G

[3, 0, 1]


array([[[1, 1, 0, 1],
        [1, 1, 1, 1],
        [0, 1, 0, 1],
        [1, 1, 1, 1]],

       [[1, 1, 1, 1],
        [1, 1, 0, 1],
        [1, 0, 0, 1],
        [1, 1, 1, 1]],

       [[0, 1, 0, 1],
        [1, 0, 0, 1],
        [0, 0, 0, 0],
        [1, 1, 0, 0]],

       [[1, 1, 1, 1],
        [1, 1, 1, 1],
        [1, 1, 0, 0],
        [1, 1, 0, 1]]])

In [14]:
# vector storing the flattened list of vertices
V = res[1]
V

array([13, 13,  3, 13])

In [15]:
# np.flip(np.argsort(V))

In [16]:
# remove_edges(G, n, V, V[0])

In [17]:
G = res[0]
G

array([[[1, 1, 0, 1],
        [1, 1, 1, 1],
        [0, 1, 0, 1],
        [1, 1, 1, 1]],

       [[1, 1, 1, 1],
        [1, 1, 0, 1],
        [1, 0, 0, 1],
        [1, 1, 1, 1]],

       [[0, 1, 0, 1],
        [1, 0, 0, 1],
        [0, 0, 0, 0],
        [1, 1, 0, 0]],

       [[1, 1, 1, 1],
        [1, 1, 1, 1],
        [1, 1, 0, 0],
        [1, 1, 0, 1]]])

In [18]:
clique_vertices = res[-1]
clique_vertices

[3, 0, 1]

In [19]:
clique = False
while not clique:
    curr = np.flip(np.argsort(V))[0]
    print(curr)
    A = remove_edges(G, n, V, curr)
    G = A

    clique = is_clique(G, V, n)

In [ ]:
clique